In [1]:
import os
from openai import OpenAI, pydantic_function_tool
from dotenv import load_dotenv
import rich
import requests
import json
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

**Using Pydantic generated function structure to send in Chat API and Responses API**

The Pydantic-generated function structure is acceptable in OpenAI's Chat API, but the Responses API requires a slightly different structure.

In [4]:
class GetWeather(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")
    
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

# Notice the function property in the output, which is not acceptable in Responses API
rich.print(pydantic_function_tool(GetWeather)) 

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

First Step where model will responed with tool call request

In [8]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
    {"role": "user", "content": "Berlin"}
]
# Except for this line, everything else is the same as in the previous example
tools = [pydantic_function_tool(GetWeather)] 
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools = tools
)

rich.print(response.choices[0])
print("Finish Reason = ", response.choices[0].finish_reason)
rich.print(response.choices[0].message.tool_calls)
# rich.print(response)


Choice(
    finish_reason='tool_calls',
    index=0,
    logprobs=None,
    message=ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_GlQHIKypjRFcaKx4ssGsRZMJ',
                function=Function(arguments='{"latitude":52.52,"longitude":13.405}', name='GetWeather'),
                type='function'
            )
        ]
    )
)

Finish Reason =  tool_calls


[
    ChatCompletionMessageToolCall(
        id='call_GlQHIKypjRFcaKx4ssGsRZMJ',
        function=Function(arguments='{"latitude":52.52,"longitude":13.405}', name='GetWeather'),
        type='function'
    )
]

Second Step where we are calling the `get_weather` function and sending the response back to Chat API

In [9]:
if response.choices[0].finish_reason == "tool_calls": # Check if finish_reason is tool_calls
    tool_call = response.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = {
        "role": "tool",
        "content": json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
        "tool_call_id": tool_call.id
    }
    # Important: we will append the previous message (response.choices[0].message)
    messages.append(response.choices[0].message) 
    messages.append(new_message)
    response2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("Model Response2 = ",response2.choices[0].message.content)
    print("Finish Reason = ",response2.choices[0].finish_reason)

get_weather function called to get weather for latitude = 52.52, longitude = 13.405
And result is  = 7.5
Model Response2 =  The current weather in Berlin is approximately 7.5°C. If you need more specific details about the weather conditions, feel free to ask!
Finish Reason =  stop


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

In [10]:
rich.print(pydantic_function_tool(GetWeather))

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

To use pydantic-generated function, we need to use `openai.responses.parse()` function call

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs_tools.py

### Using old way of sending history messages in every call

First Step where model will responed with tool call request

In [12]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
    {"role": "user", "content": "Berlin"}
]
tools = [pydantic_function_tool(GetWeather)]

# Note the parse function
response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)
rich.print(response.output[0].parsed_arguments)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        id='fc_67e234cc91f081929b2fda7aae7525430ec6d6be56c9acd3',
        arguments='{"latitude":52.52,"longitude":13.405}',
        call_id='call_gJJgN88OJlgerTivy4cRferX',
        name='GetWeather',
        type='function_call',
        status='completed',
        parsed_arguments=GetWeather(latitude=52.52, longitude=13.405)
    )
]

GetWeather(latitude=52.52, longitude=13.405)

Second Step where we are calling the `get_weather` function and sending the response back to Responses API

To send the result of function call we need to send specific format object into Responses API call

Note: object has different property names.

```
{
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": <output of function call>,
}
```

In [13]:
if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    # arguments = json.loads(tool_call.arguments) # Not needed but still works
    # latitude = arguments.get("latitude")
    # longitude = arguments.get("longitude")
    # weather = get_weather(**arguments) # Not needed but still works
    
    # As we are receiving parsed_arguments as object, we can call properties on parsed_arguments object
    weather = get_weather(tool_call.parsed_arguments.latitude, tool_call.parsed_arguments.longitude) # Both will work
    
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(weather)
        # Because of json object in output Responses API sometimes does not generate expected output
        # "output":  json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
    }
    # To call use Responses API with history messages we need to send output back but 
    # it gives an error if we append tool call with parsed_arguments
    del response.output[0].parsed_arguments
    # Important: we will append the tool call (response.output[0]) 
    messages.append(response.output[0]) 
    messages.append(new_message)
    # rich.print(messages)
    # Calling the Responses API again with all the history messages and the new message
    response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)

get_weather function called to get weather for latitude = 52.52, longitude = 13.405
And result is  = 7.4
Model Response2 =  The current temperature in Berlin is 7.4°C. If you need more specific weather details or forecasts, feel free to ask!
Status =  completed


### Using new way of conversation state by sending perivous reponse id

First Step where model will responed with tool call request

In [17]:
# This section is same as above

messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
    {"role": "user", "content": "Berlin"}
]
tools = [pydantic_function_tool(GetWeather)]

response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        id='fc_67e2375de7a48192899d9f12c200ab91034353360b6b83d3',
        arguments='{"latitude":52.520008,"longitude":13.404954}',
        call_id='call_i5KkYcqG0bHZoThdT82txbGs',
        name='GetWeather',
        type='function_call',
        status='completed',
        parsed_arguments=GetWeather(latitude=52.520008, longitude=13.404954)
    )
]

Second Step where we are calling the `get_weather` function and sending the response back to Responses API

The only difference in below section is how messages are sent.

In [ ]:
# The only difference in this section is how messages are sent.

if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    # arguments = json.loads(tool_call.arguments) # Not needed but still works
    # latitude = arguments.get("latitude")
    # longitude = arguments.get("longitude")
    # weather = get_weather(**arguments) # Not needed but still works
    
    weather = get_weather(tool_call.parsed_arguments.latitude, tool_call.parsed_arguments.longitude) # Both will work
    
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(weather)
    }
    # Not needed now because we are using response.id
    # del response.output[0].parsed_arguments
    
    # Not needed now because we are using response.id 
    # messages.append(response.output[0]) 

    # Emptying the messages array because we are sending the previous response id, 
    # therefore we don't need to send the previous message
    messages = []
    messages.append(new_message)
    response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools, previous_response_id=response.id)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)

get_weather function called to get weather for latitude = 52.520008, longitude = 13.404954
And result is  = 7.4
Model Response2 =  The current temperature in Berlin is 7.4°C. If you need more details about the weather, like humidity or forecast, just let me know!
Status =  completed
